# Parsing Text (aka Prepping Text Data)
In this lesson we'll take our acquired data and parse it, that is, we'll better understand the text data by *breaking it down into smaller components.

We'll be using the `nltk` package, which requires a little bit of up-front setup:

In [ ]:
# We don't need to install nltk, it should come with anaconda, but nltk needs to download data.
# python -c "import nltk; nltk.download('stopwords')"

In [ ]:
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd

from time import strftime

Here's our plan for parsing the text data:

1. Convert text to all lower case for normalcy.
1. Remove any accented characters, non-ASCII characters.
1. Remove special characters.
1. Tokenize the words.
1. Stem or lemmatize the words.
1. Remove stopwords.
1. Store the clean text and the original text for use in future notebooks

In [ ]:
original = '''
What are the Math and Stats Principles You Need for Data Science?Oct 21, 2020 | Data Science


Coming into our Data Science program, you will need to know some math and stats. However, many of our applicants actually learn in the application process – you don’t need to be an expert before applying! Data science is a very accessible field to anyone dedicated to learning new skills, and we can work with any applicant to help them learn what they need to know. But what “skills” do we mean, exactly? Just what exactly are the data science math and stats principles you need to know?
What are the main math principles you need to know to get into Codeup’s Data Science program?


Algebra
Do you know PEMDAS and can you solve for x? You will need to be or become comfortable with the following:

Variables (x, y, n, etc.)
Formulas, functions, and variable manipulations (e.g. x^2 = x + 6, solve for x).
Order of evaluation: PEMDAS: parentheses, exponents, then multiplication, division, addition, and subtraction
Commutativity where a + b = b + a
Associativity where a + (b + c) = (a + b) + c
Adding and subtracting matrices
A conceptual understanding of exponential growth/decay- things can increase at an increasing rate

Descriptive Statistics
Know what a min, max, mode, median, and average are. Have a conceptual understanding that stats/probability is about trying to quantify uncertainty.
Data Visualization
Know what a scatterplot is and how to read a barplot.
How to Learn and Expand on These Concepts
There are a number of great resources out there to teach you these and similar concepts. Khan Academy is a great starting place for data science math! If you want to know what exactly we assign our applicants, you’ll just have to apply!
 
What about once you’re in Codeup?


What You Won’t Do
Do we do any mathematical proofs for concepts or perform derivations? No. 
Do we do any calculus and probability calculating by hand? No.
Are we transforming equations, where we cancel out units or terms and do lots of algebraic gymnastics? No
What You Will Do
Will we have Python solve our linear algebra problems for us? Yes
Will we have Python calculate probabilities, the area under a curve, and the slope of a line for us? Yes
Will we have Python do all of the calculus for us? Yes
 
See, the data science math and stats slice of the pie is certainly doable. If you like problem-solving and are ready to challenge yourself, you’ll love data science! If you are interested in learning about data science, just apply! Our Admissions Manager can work with you to get you where you need to be starting from where you are now. Let us help you get there so you can launch a great new career.

Request More Info





Our ProgramsFull Stack Web Development
Data Science
Cyber Cloud
Systems Engineering





Latest Blog Articles
Codeup Dallas Open House
Codeup’s Placement Team Continues Setting Records
IT Certifications 101: Why They Matter, and Why They Don’t
A rise in cyber attacks means opportunities for veterans in San Antonio
Use your GI Bill® benefits to Land a Job in Tech























More From This Category






Codeup Dallas Open House
Nov 30, 2021 | Dallas Newsletter, EventsCome join us for the re-opening of our Dallas Campus with some drinks and snacks at Codeup! Curious about what our campus looks like? Click here to register for free About this event Come join us for the re-opening of our Dallas Campus with some drinks and snacks at...




Codeup’s Placement Team Continues Setting Records
Nov 19, 2021 | Codeup News, EmployersOur Placement Team is simply defined as a group that manages relationships with our employer partners and our graduating students to help get our graduating students hired. Last quarter the Placement Team helped 48 students get hired to life-changing careers in tech....




IT Certifications 101: Why They Matter, and Why They Don’t
Nov 18, 2021 | Cybersecurity, IT Training, Tips for Prospective StudentsAWS, Google, Azure, Red Hat, CompTIA...these are big names in IT! And not only for their products, but also for the certifications they offer. If you’re new to tech, you might be wondering: Do certifications really matter? Welcome to IT Certifications 101! What’s the...

'''

#### Step 1: Convert all text to lower for normalcy

In [ ]:
article = original.lower()
print(article)

#### Step 2: Remove any accented characters, non-ASCII characters.

Usually in any text corpus, you might be dealing with accented characters/letters, especially if you only want to analyze the English language. Hence, we need to make sure that these characters are converted and standardized into ASCII characters. A simple example is converting é to e.

We'll go about this in three steps:

1. `unicodedata.normalize` removes any inconsistencies in unicode character encoding.
1. `.encode` to convert the resulting string to the ASCII character set. We'll ignore any errors in conversion, meaning we'll drop anything that isn't an ASCII character.
1. `.decode` to turn the resulting bytes object back into a string.

In [ ]:
article = unicodedata.normalize('NFKD', article)\
    .encode('ascii', 'ignore')\
    .decode('utf-8', 'ignore')

print(article)

#### Step 3: Remove special characters

Special characters and symbols are usually non-alphanumeric characters or even occasionally numeric characters (depending on the problem), which add to the extra noise in unstructured text. Usually, simple regular expressions (regexes) can be used to remove them.

In [ ]:
# remove anything that is not a through z, a number, a single quote, or whitespace
article = re.sub(r"[^a-z0-9'\s]", '', article)
print(article)

#### Step 4: Tokenization

After removing non-ASCII characters and special characters, it's common to tokenize the strings, to break words and any punctuation left over into discrete units. Tokenization is the process of breaking something down into discrete units. In the context of NLP, this means breaking text down into discrete words, punctuation, etc.

We will use `nltk` to do tokenization for us:

In [ ]:
tokenizer = nltk.tokenize.ToktokTokenizer()

print(tokenizer.tokenize(original, return_str=True))

#### Step 5: Stemming and Lemmatization

Usually you will want to use lemmatization. We will demonstrate why that is the case by looking at both here.

#### Stemming:
Word stems are the base form of a word.

We create new words by attaching affixes in a process known as inflection. For example, "calls", "called", and "calling" all share the base stem "call".

The Porter stemmer is based on the algorithm developed by its inventor, Dr. Martin Porter. Originally, the algorithm is said to have had a total of five different phases for reduction of inflections to their stems, where each phase has its own set of rules.

Note that usually stemming has a fixed set of rules, hence, the root stems may not be lexicographically correct. This means that the stemmed words may not be semantically correct, and might have a chance of not being present in the dictionary (as we'll see in the output of stemming).

In [ ]:
# Create the nltk stemmer object, then use it
ps = nltk.porter.PorterStemmer()

ps.stem('call'), ps.stem('called'), ps.stem('calling')

Now we can apply this stemming transformation to all the words in the article.

In [ ]:
stems = [ps.stem(word) for word in article.split()]
article_stemmed = ' '.join(stems)
print(article_stemmed)

In [ ]:
pd.Series(stems).value_counts().head(10)

#### Lemmatization
Lemmatization is very similar to stemming, however, the base form in this case is known as the root word, but not the root stem. The difference is that the root word is always a lexicographically correct word (present in the dictionary), but the root stem may not be so. Thus, root word, also known as the lemma, will always be present in the dictionary.

Note that the lemmatization process is considerably slower than stemming, because an additional step is involved where the root form or lemma is formed by removing the affix from the word if and only if the lemma is present in the dictionary.

Let's take a look at a simple example of the difference between stemming and lemmatization:

In [ ]:
# python3 -c "import nltk; nltk.download('all')"
# OR
# python3 -c "import nltk; nltk.download('wordnet')"

In [ ]:
wnl = nltk.stem.WordNetLemmatizer()

for word in 'study studies'.split():
    print('stem:', ps.stem(word), '-- lemma:', wnl.lemmatize(word))

And now we can apply lemmatization to our entire document:

In [ ]:
lemmas = [wnl.lemmatize(word) for word in article.split()]
article_lemmatized = ' '.join(lemmas)

print(article_lemmatized)

Now that we have a list of the lemmas, we can take a look at the most frequent words.

In [ ]:
pd.Series(lemmas).value_counts()[:10]

#### Step 6: Removing stopwords

Words which have little or no significance, especially when constructing meaningful features from text, are known as stop words (or stopwords). These are usually words that end up having the maximum frequency if you do a simple term or word frequency in a corpus. Typically, these can be articles, conjunctions, prepositions and so on. Some examples of stopwords: a, an, the, and like.

While there is no universal stopword list, we will use a standard English language stopwords list from nltk. You can also add your own domain-specific stopwords as needed.

Before removing stopwords, we want to segment text into linguistic units such as words or numbers. This process is called tokenization.

In [ ]:
stopword_list = stopwords.words('english')

stopword_list.remove('no')
stopword_list.remove('not')

stopword_list[:10]

In [ ]:
words = article.split()
filtered_words = [w for w in words if w not in stopword_list]

print('Removed {} stopwords'.format(len(words) - len(filtered_words)))
print('---')

article_without_stopwords = ' '.join(filtered_words)

print(article_without_stopwords)

## Exercises
The end result of this exercise should be a file named `prepare.py` that defines the requested functions.

In this exercise we will be defining some functions to prepare textual data. These functions should apply equally well to both the codeup blog articles and the news articles that were previously acquired.

1. Define a function named basic_clean. It should take in a string and apply some basic text cleaning to it:

- Lowercase everything
- Normalize unicode characters
- Replace anything that is not a letter, number, whitespace or a single quote.

2. Define a function named `tokenize`. It should take in a string and tokenize all the words in the string.

3. Define a function named `stem`. It should accept some text and return the text after applying stemming to all the words.

4. Define a function named `lemmatize`. It should accept some text and return the text after applying lemmatization to each word.

5. Define a function named `remove_stopwords`. It should accept some text and return the text after removing all the stopwords.

    This function should define two optional parameters, extra_words and exclude_words. These parameters should define any additional stop words to include, and any words that we don't want to remove.
    

6. Use your data from the acquire to produce a dataframe of the news articles. Name the dataframe `news_df`.

7. Make another dataframe for the Codeup blog posts. Name the dataframe `codeup_df`.

8. For each dataframe, produce the following columns:
- `title` to hold the title
- `original` to hold the original article/post content
- `clean` to hold the normalized and tokenized original with the stopwords removed.
- `stemmed` to hold the stemmed version of the cleaned data.
- `lemmatized` to hold the lemmatized version of the cleaned data.

Ask yourself:

If your corpus is 493KB, would you prefer to use stemmed or lemmatized text?
If your corpus is 25MB, would you prefer to use stemmed or lemmatized text?
If your corpus is 200TB of text and you're charged by the megabyte for your hosted computational resources, would you prefer to use stemmed or lemmatized text?